# 実験に利用するデータを用意する

実験に利用するデータを用意する。

#### 1-3-1. AWS S3ストレージからデータを用意する

In [ ]:
import os
import sys
sys.path.append('../../../../../..')
from library.experiment.prepare_data import DataPreparer
DataPreparer(working_path=os.path.abspath('__file__')).from_AWS()

#### 1-3-2. Githubからデータを用意する

In [ ]:
import os
import sys
sys.path.append('../../../../../..')
from library.experiment.prepare_data import DataPreparer
DataPreparer(working_path=os.path.abspath('__file__')).from_github()

## 2. GRDMに実験（準備）データを同期

In [ ]:
import os
import sys
sys.path.append('../../../../../..')
from library.experiment.prepare_data import DataPreparer
DataPreparer(working_path=os.path.abspath('__file__')).completed_task()

## サブフローメニューへ

サブフローメニューへアクセスするボタンを表示します

In [ ]:
import os
import sys
sys.path.append('../../../../../..')
from library.experiment.prepare_data import DataPreparer
DataPreparer(working_path=os.path.abspath('__file__')).return_subflow_menu()